In [ ]:
import numpy as np
import numpy as np

from net import SignPredictor
from activation_functions import der_relu, relu, sigmoid
from loss_functions import binary_entropy_loss
from optimizer import Optimizer

In [2]:
x_l = np.load('../data/data/X.npy')
Y_l = np.load('../data/data/Y.npy')
# Join the input images that display zeros and ones along the row axis.
# Results in X with indices 0 to 204 being zero sign examples and indices 205 to 410 being one sign examples.
X = np.concatenate((x_l[204:409], x_l[822:1027] ), axis=0)
Y = np.concatenate((np.zeros(205), np.ones(205)), axis=0).reshape(X.shape[0],1)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]
print("Number of samples in the training data set: " , number_of_train)
print("Number of samples in the test data set: " , number_of_test)
X_train = X_train.reshape(number_of_train, X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(number_of_test, X_test.shape[1]*X_test.shape[2])

print("x train: ", X_train.shape)
print("x test: ", X_test.shape)
print("y train: ", Y_train.shape)
print("y test: ", Y_test.shape)

Number of samples in the training data set:  348
Number of samples in the test data set:  62
x train:  (348, 4096)
x test:  (62, 4096)
y train:  (348, 1)
y test:  (62, 1)


In [12]:
X_train.shape[0]/batch_size

NameError: name 'batch_size' is not defined

In [14]:
total_epochs = 2000
batch_size = 64
batches_per_epoch = np.ceil(X_train.shape[0]/batch_size)
optimizer = Optimizer(
    lr = 0.03,
    adam_args = {
        "beta_1": 0.9,
        "beta_2": 0.999,
        "epsilon": 1e-8,
    },
    lr_decay_type= "cosine",
    total_epochs = total_epochs * batches_per_epoch
)

net = SignPredictor(
    input_size = X_train.shape[1],
    layer_sizes= [10, 1],
    activations = [relu, sigmoid],
    activations_der=[der_relu, None],
    loss_fn = binary_entropy_loss,
    loss_fn_der= None,
    optimizer=optimizer
)

In [16]:
t = 1
for epoch in range(total_epochs):
    # Introduce batching:
    perm = np.random.permutation(X_train.shape[0])
    X_train_shuffled = X_train[perm]
    Y_train_shuffled = Y_train[perm]
    epoch_loss = 0
    
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train_shuffled[i:i+batch_size]
        Y_batch = Y_train_shuffled[i:i+batch_size]
        loss, pred = net.train_step(X_batch, Y_batch, t)
        t += 1
        epoch_loss += loss * X_batch.shape[0]  # Weight by batch size
    
    epoch_loss /= X_train.shape[0]

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {epoch_loss:.4f}, learning rate: {optimizer.current_lr}")

print(f"Final Epoch prediction: {pred.round()}, target: {Y_train}")

test_pred = net.predict(X_test)
print(f"Share of correctly predicted images {np.mean(test_pred.round() == Y_test)}")


Epoch 0, Loss: 0.0031, learning rate: 0.02999998149449555
Epoch 100, Loss: 0.0012, learning rate: 0.02981162093252581
Epoch 200, Loss: 0.0006, learning rate: 0.0292585491062457
Epoch 300, Loss: 0.0003, learning rate: 0.028354384479558386
Epoch 400, Loss: 0.0002, learning rate: 0.027121390586323266
Epoch 500, Loss: 0.0001, learning rate: 0.025589927828263918
Epoch 600, Loss: 0.0001, learning rate: 0.023797705901117253
Epoch 700, Loss: 0.0000, learning rate: 0.021788855256773182
Epoch 800, Loss: 0.0000, learning rate: 0.019612840465088512
Epoch 900, Loss: 0.0000, learning rate: 0.017323242232013567
Epoch 1000, Loss: 0.0000, learning rate: 0.014976438064787535
Epoch 1100, Loss: 0.0000, learning rate: 0.012630214070604018
Epoch 1200, Loss: 0.0000, learning rate: 0.010342342070870814
Epoch 1300, Loss: 0.0000, learning rate: 0.008169157067233203
Epoch 1400, Loss: 0.0000, learning rate: 0.006164170086868255
Epoch 1500, Loss: 0.0000, learning rate: 0.004376750563402862
Epoch 1600, Loss: 0.0000